In [1]:
print("Welcome to FFM Slack Bot!")

Welcome to FFM Slack Bot!


In [2]:
#Importing the necessary libraries
import os
from dotenv import load_dotenv #This is used to load the .env file
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# MODEL = "gpt-3.5-turbo" #This is a chat model 
# MODEL = "llama3"
MODEL = "llama2"

In [13]:

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY, model= MODEL)
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    model.invoke("Tell me a joke")
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()
model.invoke("Tell me a joke")
    

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that made you smile!"

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a funny joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("All About FFM.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'All About FFM.pdf', 'page': 0}, page_content="All About Fulfillment Monitor\nWhat is Fulfillment Monitor?\nFulfillment Monitor Slides.pptx\nWho do we support and what are our personas?\nGlad you asked!\nThis small but mighty development team supports requests from Suppliers, Retailers/Outsourced \nRetailers (OSRs), and 3PL. The most common persona that we have found to use Fulfillment Monitor is \nour CIO/technologist but since it's easy to use, other personas find value in what we offer.\nPersonas who leverage Fulfillment Monitor\nTable of Contents\nWhat is Fulfillment Monitor?\nWho do we support and \nwhat are our personas?\nPersonas who \nleverage \nFulfillment Monitor\nWhat Fulfillment Monitor is \nnot?\nTraining\nSupplemental \nDocumentation\nFAQ\nInternal Access\nFulfillment Monitor\nVisibility\nExternal Access\nOrg Access - FFM\nUser Access - \nFFM\nOrg Access - \nVisibility\nUser Access - \nVisibility\nCompany Administrator\nGlossary\nTransactions

In [16]:
from langchain.prompts import PromptTemplate

# create a prompt template 
template = """
Answer the question based on the context below. If you cant 
answer the question, reply " Sorry, I don't know".

context: {context}

question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you cant \nanswer the question, reply " Sorry, I don\'t know".\n\ncontext: Here is some context\n\nquestion: Here is a question\n'

In [17]:
chain = prompt | model | parser
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [19]:
chain.invoke(
    {
    "context":"My name is Kelechi and I am a software developer", 
     "question":"what is my name and what is my occupation?"
     }
             )

' Of course! Based on the context provided, your name is Kelechi and your occupation is Software Developer.'

In [20]:
from langchain_community.vectorstores import DocArrayInMemorySearch
# create a vector store loading the pages in memory. for a real use case, you would use a vector store that is stored on disk
vector_store = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [21]:
retriever  = vector_store.as_retriever()
retriever.invoke("Fulfillment Monitor")

[Document(metadata={'source': 'All About FFM.pdf', 'page': 1}, page_content='1.  \n2.  \na.  \n3.  \nWhat Fulfillment Monitor is not?\nFFM will not have exact parity with some features that Transaction Tracker has. This is due to FFM being \na customer facing product to simplify the data that SPS employees have access to.\nTraining\nWithin the Training Center , search for Fulfillment Monitor for training and webinars\nSupplemental Documentation\nFulfillment Monitor User Guide\nScheduled Reports  Info\nDelivery Details\nRedelivery\nMark As Resolved\nFAQ\nCurrent FAQ sheet: Fulfillment+Monitor+-+FAQ.pdf\nWhat Languages does FFM Support?\nEnglish, French, French Canadian\nHow do I remove Partner Names that my customer no longer does business with?Run Report Now\nCreating a new Scheduled \nReport\nFTP Delivery\nFile Formats for \nFTP Delivery\nSaved Searches\nAdvanced Search\nDocument Settings\nDownload \nPreferences\nError Automation\nException Settings\nSet Default \nTimeline for \nExcep

In [23]:
from operator import itemgetter

chain = (
    {
        "context":itemgetter("question") | retriever, 
        "question":itemgetter("question")
    }
    | prompt 
    | model 
    | parser
)

chain.invoke({"question":"What is the Fulfillment Monitor?"})

"Sorry, I don't know. The document you provided does not contain any information about what the Fulfillment Monitor is. It mentions it in various contexts, but none of the passages provide a clear definition or explanation of the term. Can you please provide more context or clarify your question?"

In [24]:
questions = [
    "How can I get internal Access to the system?",
    "What is the Fulfillment Monitor?", 
    "What is the process for onboarding new employees?",
    "How can I give a Retailer Organization account access to see Fulfillment Monitor",
    "Who do we support and what are our personas?"
    
    ]

for question in questions:
    print(f"Question:, {question}")
    print(f"Answer: {chain.invoke({'question':question})}")
    print("\n")                

Question:, How can I get internal Access to the system?
Answer: Sorry, I don't know the answer to your question based on the provided context. The document provides information about Fulfillment Monitor, its features, and how to use it, but it does not mention anything about getting internal access to the system. If you have any other questions or need further clarification, please let me know.


Question:, What is the Fulfillment Monitor?
Answer:  Based on the provided document, Fulfillment Monitor appears to be a tool that provides visibility and control over fulfillment processes for businesses. It allows users to manage transactions, create and edit saved searches, download reports, and more. The tool also has features such as exception settings, error automation, and communication enhancement requests. Additionally, it seems to support multiple languages and has a limit on the number of scheduled reports that can be created.

Sorry, I don't know the specific answer to your questio